In [1]:
!date

import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az
import xarray as xr
import theano.tensor as tt
import matplotlib.pyplot as plt

%matplotlib inline

lun jun  8 05:10:48 CEST 2020


In [2]:
az.rcParams["stats.ic_pointwise"] = True

In [3]:
df_all = pd.read_csv("18-19_table.txt", sep="\t", comment="#")
team_names = df_all["Home \ Away "].str.strip().copy(deep=True).values
team_abbrs = df_all.columns.str.strip().copy(deep=True).values[1:]
df_all = df_all.melt(id_vars=["Home \ Away "])
df_all.columns = ["home_team", "away_team", "result"]
df_all = df_all.apply(lambda x: x.str.strip(), axis="index")
df_all = df_all.replace(
    {
        "away_team": {abbr: team for abbr, team in zip(team_abbrs, team_names)},
        "result": {"—": np.nan}
    }
).dropna(subset=["result"]).reset_index(drop=True)
df_aux = df_all.result.str.split("–", expand=True)
df_aux.columns = ["home_goals", "away_goals"]
df_final = pd.concat((df_all[["home_team", "away_team"]], df_aux), axis="columns")
df_final.to_csv("18-19_df.csv", index=False)

In [4]:
df = pd.read_csv("18-19_df.csv")
home_team_idxs, team_names = pd.factorize(df.home_team, sort=True)
away_team_idxs, _ = pd.factorize(df.away_team, sort=True)
num_teams = len(team_names)
df

,home_team,away_team,home_goals,away_goals
0,Bournemouth,Arsenal,1,2
1,Brighton & Hove Albion,Arsenal,1,1
2,Burnley,Arsenal,1,3
3,Cardiff City,Arsenal,2,3
4,Chelsea,Arsenal,3,2
...,...,...,...,...
375,Newcastle United,Wolverhampton Wanderers,1,2
376,Southampton,Wolverhampton Wanderers,3,1
377,Tottenham Hotspur,Wolverhampton Wanderers,1,3
378,Watford,Wolverhampton Wanderers,1,2


In [5]:
with pm.Model() as m_general:
    # constant data
    home_team = pm.intX(pm.Data("home_team", home_team_idxs))
    away_team = pm.intX(pm.Data("away_team", away_team_idxs))
    
    # global model parameters
    home = pm.Normal('home', mu=0, sigma=5)
    sd_att = pm.HalfStudentT('sd_att', nu=3, sigma=2.5)
    sd_def = pm.HalfStudentT('sd_def', nu=3, sigma=2.5)
    intercept = pm.Normal('intercept', mu=0, sigma=5)

    # team-specific model parameters
    atts_star = pm.Normal("atts_star", mu=0, sigma=sd_att, shape=num_teams)
    defs_star = pm.Normal("defs_star", mu=0, sigma=sd_def, shape=num_teams)

    atts = pm.Deterministic('atts', atts_star - tt.mean(atts_star))
    defs = pm.Deterministic('defs', defs_star - tt.mean(defs_star))
    home_theta = tt.exp(intercept + home + atts[home_team] + defs[away_team])
    away_theta = tt.exp(intercept + atts[away_team] + defs[home_team])

    # likelihood of observed data
    home_goals = pm.Poisson('home_goals', mu=home_theta, observed=df.home_goals)
    away_goals = pm.Poisson('away_goals', mu=away_theta, observed=df.away_goals)

In [6]:
dims = {
    "home_goals": ["match"],
    "away_goals": ["match"],
    "home_team": ["match"],
    "away_team": ["match"],
    "atts": ["team"],
    "atts_star": ["team"],
    "defs": ["team"],
    "defs_star": ["team"],
}
coords = {"team": team_names}
with m_general:
    trace = pm.sample(random_seed=1375)
    idata_general = az.from_pymc3(trace, coords=coords, dims=dims)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [defs_star, atts_star, intercept, sd_def, sd_att, home]
Sampling 4 chains, 0 divergences: 100%|██████████| 4000/4000 [00:02<00:00, 1685.93draws/s]


In [7]:
idata_general.log_likelihood

<xarray.Dataset>
Dimensions:     (chain: 4, draw: 500, match: 380)
Coordinates:
  * chain       (chain) int64 0 1 2 3
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
  * match       (match) int64 0 1 2 3 4 5 6 7 ... 373 374 375 376 377 378 379
Data variables:
    home_goals  (chain, draw, match) float64 -1.037 -1.082 ... -1.0 -1.088
    away_goals  (chain, draw, match) float64 -1.448 -1.505 ... -1.426 -1.02
Attributes:
    created_at:                 2020-06-08T03:11:00.137861
    arviz_version:              0.8.3
    inference_library:          pymc3
    inference_library_version:  3.8

In [8]:
# define helpers to make answer less verbose
log_lik = idata_general.log_likelihood
const = idata_general.constant_data

In [9]:
log_lik["matches"] = log_lik.home_goals + log_lik.away_goals
az.loo(idata_general, var_name="matches")

Computed from 2000 by 380 log-likelihood matrix

         Estimate       SE
elpd_loo -1100.95    16.87
p_loo       29.11        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)      380  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%


The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if
you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive
accuracy.

In [10]:
home_goals_team = log_lik.home_goals.groupby(const.home_team).sum().rename({"home_team": "team"})
away_goals_team = log_lik.away_goals.groupby(const.away_team).sum().rename({"away_team": "team"})
log_lik["teams"] = home_goals_team + away_goals_team
log_lik["team"] = team_names
az.loo(idata_general, var_name="teams")

/home/oriol/venvs/arviz-dev/lib/python3.6/site-packages/arviz/stats/stats.py:671: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


Computed from 2000 by 20 log-likelihood matrix

         Estimate       SE
elpd_loo -1103.64    30.03
p_loo       27.57        -

There has been a warning during the calculation. Please check the results.
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)        1    5.0%
 (0.5, 0.7]   (ok)          4   20.0%
   (0.7, 1]   (bad)        12   60.0%
   (1, Inf)   (very bad)    3   15.0%


The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if
you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive
accuracy.

In [11]:
log_lik["goals"] = xr.concat((log_lik.home_goals, log_lik.away_goals), "match").rename({"match": "goal"})
az.loo(idata_general, var_name="goals")

Computed from 2000 by 760 log-likelihood matrix

         Estimate       SE
elpd_loo -1100.88    17.70
p_loo       28.99        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)      760  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%


The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if
you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive
accuracy.

In [12]:
az.loo(idata_general, var_name="home_goals")

Computed from 2000 by 380 log-likelihood matrix

         Estimate       SE
elpd_loo  -567.64    11.19
p_loo       14.99        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)      380  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%


The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if
you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive
accuracy.

In [13]:
az.loo(idata_general, var_name="away_goals")

Computed from 2000 by 380 log-likelihood matrix

         Estimate       SE
elpd_loo  -533.25    13.66
p_loo       14.00        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)      380  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%


The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if
you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive
accuracy.